# RNN
---
循环神经网络（Recurrent Neural Network, RNN）是一种专门设计用于处理序列数据的神经网络。与传统的前馈神经网络不同，RNN具有内部记忆机制，能够捕捉序列数据中的时间依赖关系。
## 基本结构
RNN的核心思想是使用循环结构来处理序列数据。在每个时间步，RNN会接收一个输入（如一个词或字符），并结合前一个时间步的隐藏状态来更新当前的隐藏状态。这个隐藏状态可以看作是网络的“记忆”，它包含了之前时间步的信息。

## 数学表示

\mathbf{H}_t = \phi(\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}  + \mathbf{b}_h).

## 特点
1. 记忆机制：RNN能够通过隐藏状态保留之前时间步的信息，适用于处理具有时间依赖性的序列数据。

2. 序列处理：RNN可以处理任意长度的输入序列，输出序列的长度也可以根据任务需求进行调整。

3. 参数共享：RNN在每个时间步使用相同的权重矩阵，这使得模型参数较少，便于训练。

## 挑战
1. 梯度消失和梯度爆炸：由于RNN的循环结构，在反向传播过程中，梯度可能会随着时间步的增加而消失或爆炸，导致模型难以训练。

2. 长距离依赖问题：RNN在处理长序列时，难以捕捉远距离的时间依赖关系。

## 应用
RNN及其变体在自然语言处理、语音识别、时间序列预测等领域有广泛应用：

1. 语言模型：预测下一个词或字符的概率。

2. 机器翻译：将一种语言的句子翻译成另一种语言。

3. 语音识别：将语音信号转换为文本。

4. 时间序列预测：如股票价格预测、天气预报等。

## 困惑度（Perplexity）
在RNN（循环神经网络）中，困惑度（Perplexity）是评估语言模型的一种方式，用于量化模型生成文本的质量。具体来说，困惑度定义为模型对给定测试集的预测概率的反向度量，计算公式如下：

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>exp</mi>
  <mo data-mjx-texclass="NONE">&#x2061;</mo>
  <mrow data-mjx-texclass="INNER">
    <mo data-mjx-texclass="OPEN">(</mo>
    <mo>&#x2212;</mo>
    <mfrac>
      <mn>1</mn>
      <mi>n</mi>
    </mfrac>
    <munderover>
      <mo data-mjx-texclass="OP">&#x2211;</mo>
      <mrow data-mjx-texclass="ORD">
        <mi>t</mi>
        <mo>=</mo>
        <mn>1</mn>
      </mrow>
      <mi>n</mi>
    </munderover>
    <mi>log</mi>
    <mo data-mjx-texclass="NONE">&#x2061;</mo>
    <mi>P</mi>
    <mo stretchy="false">(</mo>
    <msub>
      <mi>x</mi>
      <mi>t</mi>
    </msub>
    <mo>&#x2223;</mo>
    <msub>
      <mi>x</mi>
      <mrow data-mjx-texclass="ORD">
        <mi>t</mi>
        <mo>&#x2212;</mo>
        <mn>1</mn>
      </mrow>
    </msub>
    <mo>,</mo>
    <mo>&#x2026;</mo>
    <mo>,</mo>
    <msub>
      <mi>x</mi>
      <mn>1</mn>
    </msub>
    <mo stretchy="false">)</mo>
    <mo data-mjx-texclass="CLOSE">)</mo>
  </mrow>
  <mo>.</mo>
</math>

困惑度的最好的理解是“下一个词元的实际选择数的调和平均数”。 我们看看一些案例。

在最好的情况下，模型总是完美地估计标签词元的概率为1。 在这种情况下，模型的困惑度为1。

在最坏的情况下，模型总是预测标签词元的概率为0。 在这种情况下，困惑度是正无穷大。

在基线上，该模型的预测是词表的所有可用词元上的均匀分布。 在这种情况下，困惑度等于词表中唯一词元的数量。 事实上，如果我们在没有任何压缩的情况下存储序列， 这将是我们能做的最好的编码方式。 因此，这种方式提供了一个重要的上限， 而任何实际模型都必须超越这个上限。

![RNN](https://zh-v2.d2l.ai/_images/rnn-train.svg "RNN")

## 简单代码实现

### 独热编码
独热编码（One-Hot Encoding）是一种常用的数据预处理技术，特别是在处理分类数据时。它将分类变量转换为二进制向量，其中只有一个元素为1，其余元素为0。这种编码方式使得分类变量可以被机器学习算法直接处理。我们每次采样的小批量数据形状是二维张量： （批量大小，时间步数）。 one_hot函数将这样一个小批量数据转换成三维张量， 张量的最后一个维度等于词表大小（len(vocab)）。 我们经常转换输入的维度，以便获得形状为 （时间步数，批量大小，词表大小）的输出。 这将使我们能够更方便地通过最外层的维度， 一步一步地更新小批量数据的隐状态。

### 初始化模型参数


In [ ]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

### 循环神经网络模型
我们首先需要一个init_rnn_state函数在初始化时返回隐状态

In [ ]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

下面的rnn函数定义了如何在一个时间步内计算隐状态和输出。

In [ ]:
def rnn(inputs, state, params):
    # inputs的形状：(时间步数量，批量大小，词表大小)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # X的形状：(批量大小，词表大小)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)

接下来我们创建一个类来包装这些函数， 并存储从零开始实现的循环神经网络模型的参数。

In [ ]:
class RNNModelScratch: #@save
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

## 预测
在循环遍历prefix中的开始字符时， 我们不断地将隐状态传递到下一个时间步，但是不生成任何输出。 这被称为预热（warm-up）期， 因为在此期间模型会自我更新（例如，更新隐状态）， 但不会进行预测。

In [ ]:
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

## 梯度裁剪

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mrow data-mjx-texclass="ORD">
    <mi mathvariant="bold">g</mi>
  </mrow>
  <mo stretchy="false">&#x2190;</mo>
  <mo data-mjx-texclass="OP" movablelimits="true">min</mo>
  <mrow data-mjx-texclass="INNER">
    <mo data-mjx-texclass="OPEN">(</mo>
    <mn>1</mn>
    <mo>,</mo>
    <mfrac>
      <mi>&#x3B8;</mi>
      <mrow>
        <mo data-mjx-texclass="ORD" fence="false" stretchy="false">&#x2016;</mo>
        <mrow data-mjx-texclass="ORD">
          <mi mathvariant="bold">g</mi>
        </mrow>
        <mo data-mjx-texclass="ORD" fence="false" stretchy="false">&#x2016;</mo>
      </mrow>
    </mfrac>
    <mo data-mjx-texclass="CLOSE">)</mo>
  </mrow>
  <mrow data-mjx-texclass="ORD">
    <mi mathvariant="bold">g</mi>
  </mrow>
  <mo>.</mo>
</math>

In [ ]:
def grad_clipping(net, theta):  #@save
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm